# Importing Data from Big Query and Pivoting

## Raison d'etre

 The objective of this notebook is to <a href = "https://pbpython.com/pandas-pivot-table-explained.html"> pivot </a> the NOAA dataset's temperature and precipitation elements for ALL weather recordings of it's weather stations from the year 1900 to the present. Each of the steps required to do this will be extensively documented for didactic reasons.

## Installation pain

 To get started on this notebook you need to be able to authenticate your Big Query account with google.
 <br>
 For information on how to do this visit : https://cloud.google.com/iam/docs/quickstart-client-libraries
 <br>
 A quick summary of what you will find in the above hyperlink is as follow
 <ul>
    <li> A valid gmail account </li>
    <li> In https://console.cloud.google.com/ create a new project </li>
    <li> Using your project name 
        create a service account key. The end result of this process is the creation of a JSON file.</li>
    <li> Create an enviromental variable called <strong> GOOGLE_APPLICATION_CREDENTIALS </strong> and link that variable to the path of your JSON file </li>
    <li> Please visit the link above for the detailed information</li>
    
 </ul> 

## A Big Query of NOAA's data
 To be able to pivot the relevant data we first need to fetch it. In order to fetch it we need to know where it is located it and how it is structured. Let us inspect the project name of <strong> bigquery-public-data </strong> specifically the schema <strong> ghcn_d </strong>. The below script queries the NOAA schema and unveils all the tables corresponding to that schema.

In [36]:
from google.cloud import bigquery
from google.cloud.bigquery.job import (
    LoadJobConfig, 
    WriteDisposition, 
    CreateDisposition
)

PROJECT='fjvr-testing'  # CHANGE THIS TO YOUR PROJECT NAME

bq = bigquery.Client(project=PROJECT)

# list tables in dataset
tables = bq.list_tables("bigquery-public-data.ghcn_d")

# Iterate over all tables in the schema and returns only those that have a year as the last four characters
for table in tables:
    try:
        if int(table.table_id[-4:]) >= 1900:
            print(table.table_id)
    except:
        pass

ghcnd_1900
ghcnd_1901
ghcnd_1902
ghcnd_1903
ghcnd_1904
ghcnd_1905
ghcnd_1906
ghcnd_1907
ghcnd_1908
ghcnd_1909
ghcnd_1910
ghcnd_1911
ghcnd_1912
ghcnd_1913
ghcnd_1914
ghcnd_1915
ghcnd_1916
ghcnd_1917
ghcnd_1918
ghcnd_1919
ghcnd_1920
ghcnd_1921
ghcnd_1922
ghcnd_1923
ghcnd_1924
ghcnd_1925
ghcnd_1926
ghcnd_1927
ghcnd_1928
ghcnd_1929
ghcnd_1930
ghcnd_1931
ghcnd_1932
ghcnd_1933
ghcnd_1934
ghcnd_1935
ghcnd_1936
ghcnd_1937
ghcnd_1938
ghcnd_1939
ghcnd_1940
ghcnd_1941
ghcnd_1942
ghcnd_1943
ghcnd_1944
ghcnd_1945
ghcnd_1946
ghcnd_1947
ghcnd_1948
ghcnd_1949
ghcnd_1950
ghcnd_1951
ghcnd_1952
ghcnd_1953
ghcnd_1954
ghcnd_1955
ghcnd_1956
ghcnd_1957
ghcnd_1958
ghcnd_1959
ghcnd_1960
ghcnd_1961
ghcnd_1962
ghcnd_1963
ghcnd_1964
ghcnd_1965
ghcnd_1966
ghcnd_1967
ghcnd_1968
ghcnd_1969
ghcnd_1970
ghcnd_1971
ghcnd_1972
ghcnd_1973
ghcnd_1974
ghcnd_1975
ghcnd_1976
ghcnd_1977
ghcnd_1978
ghcnd_1979
ghcnd_1980
ghcnd_1981
ghcnd_1982
ghcnd_1983
ghcnd_1984
ghcnd_1985
ghcnd_1986
ghcnd_1987
ghcnd_1988
ghcnd_1989
ghcnd_1990

## Pivot for 2019 and the DCA Airport weather station

 In the below query we retrieve the weather observations (for the weather elements of minimum temperature, maximum temperature and precipitation.) occuring in 2019 of the DCA Ronald Reagan weather station. To do the pivot a SELF-JOIN with the table containing the data is done. Once the query has been executed it is stored in a dataframe

In [37]:
import pandas as pd

# Create a new dataframe with the required c
df = pd.DataFrame(columns=['date','id','name','precipitation_value','state','temp_max_value','temp_min_value'])

# Example query using only one year (2019) for one station (USW00013743)
query = """
SELECT DISTINCT
  base.id, 
  base.date,
  stations.name,
  stations.state,
  temp_min.value as temp_min_value,
  temp_max.value as temp_max_value,
  precipitation.value as precipitation_value

FROM {} base
INNER JOIN {} temp_min ON base.id = temp_min.id AND base.date = temp_min.date
INNER JOIN {} temp_max ON base.id = temp_max.id AND base.date = temp_max.date
INNER JOIN {} precipitation ON base.id = precipitation.id AND base.date = precipitation.date
INNER JOIN `bigquery-public-data`.ghcn_d.ghcnd_stations stations ON base.id = stations.id

WHERE temp_min.element = 'TMIN' AND temp_max.element = 'TMAX' AND precipitation.element = 'PRCP' AND stations.id = 'USW00013743'

ORDER BY base.id, base.date
"""

parameter = "`bigquery-public-data`.ghcn_d.ghcnd_2019"

query = query.format(parameter,parameter,parameter,parameter)

df_temp = bq.query(query).to_dataframe()
df.append(df_temp)

,date,id,name,precipitation_value,state,temp_max_value,temp_min_value
0,2019-01-01,USW00013743,WASHINGTON REAGAN AP,0.0,VA,178.0,67.0
1,2019-01-02,USW00013743,WASHINGTON REAGAN AP,0.0,VA,89.0,56.0
2,2019-01-03,USW00013743,WASHINGTON REAGAN AP,0.0,VA,106.0,44.0
3,2019-01-04,USW00013743,WASHINGTON REAGAN AP,10.0,VA,100.0,22.0
4,2019-01-05,USW00013743,WASHINGTON REAGAN AP,10.0,VA,122.0,78.0
5,2019-01-06,USW00013743,WASHINGTON REAGAN AP,0.0,VA,139.0,39.0
6,2019-01-07,USW00013743,WASHINGTON REAGAN AP,0.0,VA,50.0,17.0
7,2019-01-08,USW00013743,WASHINGTON REAGAN AP,61.0,VA,156.0,39.0
8,2019-01-09,USW00013743,WASHINGTON REAGAN AP,0.0,VA,122.0,6.0
9,2019-01-10,USW00013743,WASHINGTON REAGAN AP,0.0,VA,39.0,-10.0


## Inspecting the circles and the proximity to a weather station
The <strong> cbc_cleaned_usa_statid.csv </strong> contains a list of circle names (locations where an audobon bird count has occured) and the corresponding NOAA station that is closest to it. In the below script we find the unique stations AND unique dates that are of interest in our data extraction. To understand why the sets below will be produce look into <a href="https://www.w3schools.com/sql/sql_in.asp"> IN SQL operator </a>.

In [38]:
df_shell = pd.read_csv('cbc_cleaned_usa_statid.csv')

seperator = '\',\''

station_list = df_shell.loc[:, 'closest_station_id'].unique()

string_parameter_stations = "(\'" + seperator.join(station_list) + "\')"

string_parameter_stations

date_list = df_shell.loc[:,'count_date'].unique()

string_parameter_dates = "(\'" + seperator.join(date_list) + "\')"

string_parameter_dates

"('1900-12-25','1900-12-26','1901-12-25','1901-12-24','1901-12-23','1901-12-26','1902-12-26','1902-12-25','1902-12-22','1902-12-23','1902-12-29','1902-12-24','1902-12-27','1902-12-28','1902-12-01','1903-12-25','1903-12-26','1903-12-22','1903-12-24','1903-12-28','1903-12-27','1903-12-23','1904-12-25','1904-12-26','1904-12-22','1904-12-30','1904-12-28','1904-12-24','1904-12-23','1904-12-27','1904-12-21','1904-12-31','1904-12-20','1905-12-25','1905-12-26','1905-12-20','1905-12-27','1905-12-21','1905-12-23','1905-12-24','1905-12-22','1905-12-29','1905-12-28','1906-12-25','1906-12-24','1906-12-26','1906-12-28','1906-12-27','1906-12-23','1906-12-21','1906-12-29','1906-12-22','1906-12-19','1906-12-30','1906-12-18','1907-12-25','1907-12-26','1907-12-24','1907-12-28','1907-12-29','1907-12-22','1907-12-27','1907-12-21','1907-12-23','1907-12-20','1907-12-18','1908-12-24','1908-12-25','1908-12-20','1908-12-21','1908-12-22','1908-12-26','1908-12-27','1908-12-29','1908-12-23','1908-12-31','1908-12-2

## Importing the data for ALL stations and between two years
The below loops between the initial year and the final year as per the variables. In each iteration of the loop the database is hit. The result of this query is stored in a dataframe which is later saved as a CSV with a default compression of GZIP

In [39]:
# Create the date frame that will be used to accumulate the results from each year
df = pd.DataFrame(columns=['date','id','name','precipitation_value','state','temp_max_value','temp_min_value'])

# The initial date and final date that the query will use to retrieve data
initial_year = 2018
final_year = 2019

# list tables in the big query GHCN dataset
tables = bq.list_tables("bigquery-public-data.ghcn_d")

# Iterates over all tables in the bigquery-public-data.ghcn_d datasets and only takes the once that have a year as the last 4 digits.
for table in tables:
    try:
        if int(table.table_id[-4:]) >= initial_year and int(table.table_id[-4:]) <= final_year:
            # Constructs a query to flatten data. Here we are doing a SELF-JOIN (see examples online if neccesary) to extract the attributes of interest
            query = """
            SELECT DISTINCT
              base.id, 
              base.date,
              stations.name,
              stations.state,
              temp_min.value as temp_min_value,
              temp_max.value as temp_max_value,
              precipitation.value as precipitation_value

            FROM {} base
            INNER JOIN {} temp_min ON base.id = temp_min.id AND base.date = temp_min.date
            INNER JOIN {} temp_max ON base.id = temp_max.id AND base.date = temp_max.date
            INNER JOIN {} precipitation ON base.id = precipitation.id AND base.date = precipitation.date
            INNER JOIN `bigquery-public-data`.ghcn_d.ghcnd_stations stations ON base.id = stations.id

            WHERE temp_min.element = 'TMIN' AND temp_max.element = 'TMAX' AND precipitation.element = 'PRCP' AND stations.id IN {} AND base.date IN {}

            ORDER BY base.id, base.date
            """
            # This parameter is determined baased on the current year we are iterating over
            parameter = "`bigquery-public-data`.ghcn_d." + table.table_id
            
            # The format function replaces instances of {} with the corresponding sequential input
            query = query.format(parameter,parameter,parameter,parameter,string_parameter_stations, string_parameter_dates)
            
            # Queries BigQuery public data set and creates a new dataframe object
            df_temp = bq.query(query).to_dataframe()
            
            # Appends the dataframe object to accumulated data frame
            df = df.append(df_temp)
    except:
        pass

# Saves csv to the root of where this notebook. 
# THIS FILE HAS BEEN GZIP!
# WHEN YOU WANT TO OPEN IT YOU NEED TO EXTRACT IT
df.to_csv(r'flatten_data_from_' + str(initial_year) + '_to_' + str(final_year) + ".csv", compression = "gzip")
          